# Left, inner, outer, cross and fast-track joins
By the end of this lecture you will be able to:
- do left, inner, outer and cross joins between `DataFrames`
- validate joins
- do fast-track joins on sorted integer columns

In [ ]:
import polars as pl

In [ ]:
df_left = pl.DataFrame({"id": ["A", "B", "C", None], "val": [0, 1, 2, 3]})
df_left

In [ ]:
df_right = pl.DataFrame({"id": ["A", "C", None, "D"], "val": [10, 11, 12, 13]})
df_right

## Left join
In a left join we return all the rows from the left `DataFrame` and the matched rows from the right `DataFrame`. The matching happens on the basis of the join column(s). If the join column name is the same in the left and right `DataFrames` we can pass it to the `on` argument

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="left",
        coalesce=False
    )
)

Note that:
- The order of `df_left` is maintained in this left join
- The `null` `value` in the last row if `df_left` is not joined to the `null` value in `df_right`
- If the join column name(s) are not the same in both `DataFrames` then we specify `left_on` and `right_on` instead of `on`
- as we pass `coalesce=False` (which is the default) we get `id` and `id_right` join columns in the output

If we instead pass `coalesce=True` Polars coalesces the join columns `id` and `id_right` into a single `id` column (this was the default behaviour previously, personally this is what I normally want)

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="left",
        coalesce=True
    )
)

If we set `join_nulls=True` then Polars does join on `null` values

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="left", 
        coalesce=False,
        join_nulls=True
    )
)

When there are duplicate columns in both `DataFrames` Polars adds the suffix `_right` to the duplicate columns on the right by default. We can set an alternative suffix with the `suffix` argument

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="left", 
        coalesce=False,
        suffix="_r"
    )
)

## Inner joins
In an inner join we only retain the rows in both `DataFrames` where there is a matching join key

In [ ]:
(
    df_left.join(
        df_right,
        on="id",
    )
)

As for left joins the output excludes `null` values unless we set `join_nulls=True`

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        join_nulls=True)
)

## Cross join
With a cross join we get the Cartesian product of both tables - so we end up with each row of the left `DataFrame` matched with each row of the right `DataFrame` and there is no join key.

In [ ]:
(
    df_left
    .join(
        df_right, 
        how="cross")
)

## Full outer join
In a full outer join we returns all rows when there is a match in either left or right `DataFrame`

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="full")
)

In the output we see that:
- we get an `id_right` column as we have the same name for the join column in both `DataFrames`
- the first two rows have the matching keys
- we then get the two rows where there is no match in the left `DataFrame` for `null` and `D`
- we then get the two rows where there is no match in the right `DataFrame` for `null` and `B`

We can of course choose to match on the `null` values with `join_nulls=True`

In [ ]:
(
    df_left
    .join(
        df_right, 
        on="id", 
        how="full", 
        join_nulls=True
    )
)

## Full outer join with coalesced join keys
An outer-coalesce join is like: 
- an `full outer` join followed by
- a `coalesce` of the `id` and `id_right` columns into a single `id` column with the first non-`null` value

To do this we pass `coalesce=True`

In [ ]:
(
    df_left.join(
        df_right,
        on="id",
        how="full",
        coalesce=True
    )
)

We cover filtering joins (`semi` and `anti`) in a following lecture in this Section.

## Validating joins
It is easy to unintentionally lose data when joining large `DataFrames`. Polars allows you to validate your joins to ensure this isn't happening. If validation fails then Polars raises an `Exception` instead of outputting a `DataFrame`

To illustrate validation we create two new `DataFrames`

In [ ]:
df_left_valid = pl.DataFrame({"id": ["A", "B", "C", None], "val": [0, 1, 2, 3]})
df_left_valid

In [ ]:
df_right_valid = pl.DataFrame({"id": ["A", "C", None, "D"], "val": [10, 11, 12, 13]})
df_right_valid

### 1:1 validation
With a 1:1 validation we want each row to match to a unique row in the other `DataFrames`. A use case here might be when we have data about a sale in the left `DataFrame` and data about the delivery of that sale in the right `DataFrame`.

The left join below would fail validation if we try to validate with the full `DataFrame` as `D` does not map to a key in `A`. However, this join passes validation if we restrict to the common keys (and `null` which is excluded from the join)

In [ ]:
(
    df_left_valid
    .filter(pl.col("id").is_in(["A", "C", None]))
    .join(
        df_right_valid.filter(pl.col("id").is_in(["A", "C"])),
        how="left",
        on="id",
        coalesce=False,
        validate="1:1",
    )
)

### m:1 validation
With `m:1` validation we can have multiple rows with the same join key in the left `DataFrame` mapping to the same row in the right `DataFrame`. A use case here might be that we have sales data in the left `DataFrame` and metadata about the customer in the right `DataFrame`.

This example passes validation because there is only one row with `A` in the right `DataFrame`

In [ ]:
df_left_m = pl.DataFrame({"id": ["A", "A"], "val": [0, 1]})
df_right_m = pl.DataFrame({"id": ["A", "B"], "val": [10, 11]})

In [ ]:
(
    df_left_m.
    join(
        df_right_m, 
        on="id", 
        how="left", 
        coalesce=False,
        validate="m:1"
    )
)

Exercise - Change the values in one of the `DataFrames` so that it fails validation

### 1:m validation
We can also do the reverse validation where each row on the right must match to a single row on the left but multiple rows on the right can match to the same row on the left

In [ ]:
df_left_m = pl.DataFrame({"id": ["A", "B", "C"], "val": [0, 1, 2]})
df_right_m = pl.DataFrame({"id": ["A", "B"], "val": [10, 11]})

In [ ]:
(
    df_left_m
    .join(
        df_right_m, 
        on="id", 
        how="left", 
        coalesce=False,
        validate="1:m"
    )
)

### m:m validation
This is the default and in this case no checks are made.

## Joining on multiple keys and expressions
We can join on multiple columns by passing a `list` to the `on` argument. We can also use expressions within `on` to transform a column before joining on it.

We first define new left and right `DataFrames` with an additional `year` column

In [ ]:
df_left_multiple = (
    pl.DataFrame(
        {
            "id": ["A", "B", "A", "B"], 
            "year": [2020, 2020, 2021, 2021], 
            "val": [0, 1, 2, 3]
        }
    )
)
df_left_multiple

In [ ]:
df_right_multiple = (
    pl.DataFrame(
        {
            "id": ["a", "b", "a", "b"],
            "year": [2020, 2020, 2021, 2021],
            "val": [10, 11, 12, 13],
        }
    )
)

We now:
- do a join on the `id` and `year` columns and
- ensure the `id` column is uppercase for both `DataFrames`

In [ ]:
(
    df_left_multiple
    .join(
        df_right_multiple, 
        on=[pl.col("id").str.to_uppercase(), "year"], 
        how="inner",
    )
)

## Joins in lazy mode
We can do joins in lazy mode by joining on `LazyFrames` instead of `DataFrames`.

If we only need a subset of columns from the joined `DataFrames` we can follow the `join` with a `select` so that Polars only joins the necessary columns

In [ ]:
(
    df_left
    .lazy()
    .join(
        df_right.lazy(), 
        on="id", 
        how="inner"
    )
    .select("id", "val_right")
)

We can also do `join` in streaming mode for large datasets.

## Joins on sorted columns
Polars has fast-track algorithms for joins on sorted columns. We look at the effect of this in the exercises.

## Exercises

In the exercises you will develop your understanding of:
- doing a left join of two `DataFrames`
- doing an inner join of two `DataFrames`
- doing fast-track joins on sorted integer columns

### CITES Dataset
For these exercises we use an extract from a database on international trade in endangered species gathered by the CITES organisation.

This CSV has an extract of CITES trade data for 2021

In [ ]:
csv_file = "../data/cites_extract.csv"

In [ ]:
df_CITES = pl.read_csv(csv_file)
df_CITES

The `DataFrame` shows:
- the `Year` in which the trade occured
- the `Importer` and `Exporter` country in 2-digit ISO country codes
- the scientific name for the `Taxon` and
- the `Quantity` of items in the trade

We will join the trade data to the ISO country metadata in the following CSV

In [ ]:
iso_csv_file = "../data/countries_extract.csv"

In [ ]:
df_ISO = pl.read_csv(iso_csv_file)
df_ISO

This `DataFrame` has:
- `alpha-2`: the 2-letter country code
- `name`: the full name of the country
- `region`: the region of the country

### Exercise 1
For each trade record in `df_CITES` add:
- the full country name of the importer
- the region of the importer

Add to the trade records:
- the full country name of the importer
- the region of the importer

keeping only rows where we can join these values

Add:
- the full country name of the importer
- the region of the importer

keeping all rows from both `DataFrames`

Create a `DataFrame` that has all combinations of the `Taxon`,`name` and `region` columns

Returning to the `inner` join above validate that the trade records map to unique ISO metadata

Do a left join of the ISO data based on the importer (as earlier) and the same for the exporter. Ensure the `name` and `region` columns for importer and exporter are clearly distinguished in the output

### Exercise 2
In this exercise we see the effect of joins on sorted integers

We first create a pre-sorted array of `N` integers to be the join keys.

We control the `cardinality` - the number of unique join keys - with the `cardinality` variable

In [ ]:
import numpy as np

np.random.seed(0)

N = 100_000
cardinality = N // 2

We create a left-hand `DataFrame` with:
- a sorted `id` column and
- a random `values` column

We create a right-hand `DataFrame` with
- a sorted `id` column
- a metadata column (equal to the `id` column in this case)

In [ ]:
def createDataFrames(N: int, cardinality: int):
    # Create a random array with values up to cardinality and then sort it to be the `id` column
    sortedArray = np.sort(np.random.randint(0, cardinality, N,dtype=np.int64))
    df_left = pl.DataFrame({"id": sortedArray, "values": np.random.standard_normal(N)})
    # We create the right-hand `DataFrame` with the `id` column and arbitrary metadata
    df_right = pl.DataFrame(
        {"id": [i for i in range(cardinality)], "meta": [i for i in range(cardinality)]}
    )
    return df_left, df_right


df_left, df_right = createDataFrames(N=N, cardinality=cardinality)
df_left.head()

In [ ]:
df_right.head()

Check the flags if Polars knows the `id` column is sorted on the left and right `DataFrames`

In [ ]:
print(<blank>)
print(<blank>)

Time the performance for an unsorted join

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Create new `DataFrames` and tell Polars that the `id` columns are sorted

In [ ]:
df_left_sorted = (
    <blank>
)
                
df_right_sorted = (
    <blank>
)


Check the flags to see if Polars knows the `id` column is sorted on these new `DataFrames`

In [ ]:
print(<blank>)
print(<blank>)

Time the sorted join performance

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Compare performance if only the left `DataFrame` is sorted. Hint: use `df_left_sorted` and `df_right`

In [ ]:
%%timeit -n1 -r3
(
  <blank>  
)

Compare the relative performance between sorted and unsorted joins when `cardinality` is low (say `cardinality = 100`)

## Solutions

### Solution to Exercise 1
For each trade record add:
- the full country name of the importer
- the region of the importer

In [ ]:
(
    df_CITES.join(
        df_ISO, 
        left_on="Importer", 
        right_on="alpha-2", 
        how="left",
        coalesce=True
    )
)

Add to the trade records:
- the full country name of the importer
- the region of the importer

keeping only rows where we can join these values

In [ ]:
(
    df_CITES
    .join(
        df_ISO, 
        left_on="Importer", 
        right_on="alpha-2", 
        how="inner"
    )
)

Add:
- the full country name of the importer
- the region of the importer

keeping all rows from both `DataFrames`

In [ ]:
(
    df_CITES
    .join(
        df_ISO, 
        left_on="Importer", 
        right_on="alpha-2", 
        how="full"
    )
)

Create a `DataFrame` that has all combinations of the `Taxon`,`name` and `region` columns

In [ ]:
(
    df_CITES
    .select("Taxon")
    .join(
        df_ISO.select("name", "region"), 
        how="cross"
    )
)

Returning to the `inner` join above validate that the trade records map to unique ISO metadata

In [ ]:
(
    df_CITES.join(
        df_ISO, 
        left_on="Importer", 
        right_on="alpha-2", 
        how="left", 
        coalesce=True,
        validate="m:1"
    )
)

Do a left join of the ISO data based on the importer (as earlier) and the same for the exporter. Ensure the `name` and `region` columns for importer and exporter are clearly distinguished in the output

In [ ]:
(
    df_CITES.join(
        df_ISO, 
        left_on="Importer", 
        right_on="alpha-2", 
        how="left",
        coalesce=True,
        suffix="_importer"
    ).join(
        df_ISO, 
        left_on="Exporter", 
        right_on="alpha-2", 
        how="left", 
        coalesce=True,
        suffix="_exporter"
    )
)

### Solution to Exercise 2

In [ ]:
import numpy as np

np.random.seed(0)
N = 10_000_000
cardinality = N // 2
df_left, df_right = createDataFrames(N=N, cardinality=cardinality)

Check the flags to see if Polars knows the `id` column is sorted on the left and right `DataFrames`

In [ ]:
print(df_left["id"].flags)
print(df_right["id"].flags)

Time the performance for an join where the fast-track sorted algorithm is not used

In [ ]:
%%timeit -n1 -r3
(
    df_left.join(df_right,on="id")
)

To avoid confusion we create new `DataFrames` and tell Polars that the `id` columns are sorted in these new `DataFrames`

In [ ]:
df_left_sorted = df_left.with_columns(pl.col("id").set_sorted())

df_right_sorted = df_right.with_columns(pl.col("id").set_sorted())

Check to see if Polars knows the `id` columns are sorted

In [ ]:
print(df_left_sorted["id"].flags)
print(df_right_sorted["id"].flags)

Time the sorted join performance

In [ ]:
%%timeit -n1 -r3

(
    df_left_sorted.join(df_right_sorted,left_on="id",right_on="id")
)

This is much faster than the joins with the standard (non-fast track algorithms).

The fast-track algorithm can still be used even if only one of the `DataFrames` is sorted.

Compare performance if only the left `DataFrame` is sorted

In [ ]:
%%timeit -n1 -r3
(
    df_left_sorted.join(df_right,left_on="id",right_on="id")
)

There is still a benefit if just the left `DataFrame` is sorted

In [ ]:
%%timeit -n1 -r3
(
    df_left.join(df_right_sorted,left_on="id",right_on="id")
)

So there is no performance benefit from just the right `DataFrame` being sorted 

Compare the relative performance when `cardinality` is low (say `cardinality = 100`)

With low cardinality the overall joins are much faster but the differences in performances from sorting are much smaller